In [2]:
import jax
import jax.numpy as jnp
from jax import grad, jit, lax, vmap
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from collections import deque

# ===============================
# 1. Parameters
# ===============================
N, num_agents = 100, 10
dt, Gamma = 1.0, 10.0  # Increased Gamma for sharper satisfaction
alpha0, tol = 1.0, 1e-3
mu_meet = 0.5
w_v, w_w = 1.0, 20.0
maxIter, maxLineIters = 5000, 50
armijo_beta, armijo_c = 0.5, 1e-4
stall_window, stall_tol = 15, 1e-6 

x0_list = jnp.array([
    [1.5, 10, 0], [1.5, 15, 0], [11.5, 10, 0], [13.5, 10, 0], [15.5, 10, 0],
    [31.5, 10, 0], [33.5, 10, 0], [36.5, 10, 0], [47.0, 10, 0], [47.0, 15, 0]
])
O_centers = jnp.array([[6.0, 10.0], [24.0, 10.0], [42.0, 10.0]])
r_obs = jnp.array([3.162, 3.162, 3.162])
T_centers = jnp.array([jnp.array([1.5, 1.5]) + k*jnp.array([5.0, 0.0]) for k in range(num_agents)])
G_centers = jnp.array([jnp.array([1.5, 19.0]) + k*jnp.array([5.0, 0.0]) for k in range(num_agents)])
r_target, r_goal = jnp.full(num_agents, 0.8), jnp.full(num_agents, 0.8)

meeting_groups_mat = jnp.array([
    [0, 1, 2], [2, 3, -1], [0, 4, -1], [3, 4, -1], [3, 6, -1],
    [4, 5, -1], [6, 7, -1], [5, 7, -1], [5, 8, -1], [8, 9, -1], [7, 9, -1]
])

t1_low, t1_high = 11, 51
g1_low, g1_high = max(1, N-30), N
meet_low, meet_high = 0, 70

# ===============================
# 2. Differentiable Model
# ===============================
def smooth_min(vec):
    m = jnp.min(vec)
    return -(1/Gamma) * (jnp.log(jnp.sum(jnp.exp(-Gamma * (vec - m))) + 1e-6) - Gamma * m)

def smooth_max(vec):
    m = jnp.max(vec)
    return (1/Gamma) * (jnp.log(jnp.sum(jnp.exp(Gamma * (vec - m))) + 1e-6) + Gamma * m)

def step_unicycle(carry, u):
    x, y, th = carry
    next_state = jnp.array([x + dt*u[0]*jnp.cos(th), y + dt*u[0]*jnp.sin(th), th + dt*u[1]])
    return next_state, carry 

def simulate_agent_scan(x0, u_seq):
    last_state, states = lax.scan(step_unicycle, x0, u_seq.T)
    return jnp.vstack([states, last_state]).T

@jit
def total_rho(U):
    Xall = vmap(simulate_agent_scan)(x0_list, U)
    def agent_metrics(X, Tc, Gc, rt, rg):
        pos = X[:2, :].T
        d_obs = jnp.sqrt(jnp.sum((pos[:, None, :] - O_centers[None, :, :])**2, axis=-1) + 1e-9) - r_obs
        m_obs = vmap(smooth_min)(d_obs)
        d_goal = jnp.sqrt(jnp.sum((pos - Gc)**2, axis=-1) + 1e-9)
        d_target = jnp.sqrt(jnp.sum((pos - Tc)**2, axis=-1) + 1e-9)
        t_idx = jnp.arange(N+1)
        m_goal = jnp.where((t_idx >= g1_low) & (t_idx <= g1_high), rg - d_goal, -1e5)
        m_target = jnp.where((t_idx >= t1_low) & (t_idx <= t1_high), rt - d_target, -1e5)
        return smooth_min(jnp.array([smooth_min(m_obs), smooth_max(m_goal), smooth_max(m_target)]))

    rhos_agents = vmap(agent_metrics)(Xall, T_centers, G_centers, r_target, r_goal)
    
    def compute_group_rho(member_indices):
        mask = (member_indices != -1)
        safe_idx = jnp.where(mask, member_indices, 0)
        group_pos = Xall[safe_idx, :2, :] 
        diff = group_pos[:, None, :, :] - group_pos[None, :, :, :]
        dists = jnp.sqrt(jnp.sum(diff**2, axis=2) + 1e-9) 
        pair_mask = mask[:, None] * mask[None, :] * (jnp.eye(3) == 0)
        rho_at_t = mu_meet - jnp.max(jnp.where(pair_mask[:, :, None], dists, 0.0), axis=(0, 1))
        t_idx = jnp.arange(N+1)
        rho_window = jnp.where((t_idx >= meet_low) & (t_idx <= meet_high), rho_at_t, -1e5)
        return smooth_max(rho_window)

    rhos_groups = vmap(compute_group_rho)(meeting_groups_mat)
    return smooth_min(jnp.concatenate([rhos_agents, rhos_groups]))

@jit
def compute_full_objective1(U, lam):
    rho = total_rho(U)
    control_cost = jnp.sum(w_v * U[:, 0, :]**2 + w_w * U[:, 1, :]**2)
    penalty = lam * jnp.maximum(0.0, -rho)**2
    return control_cost + penalty

# Gradient of the TOTAL objective
rho_grad_fun = jit(grad(compute_full_objective1))

# ===============================
# 3. Optimization Loop
# ===============================
U = jnp.zeros((num_agents, 2, N))
rho_hist, gnorm_hist, cost_hist = [], [], []

# U = jnp.array(np.load("U_checkpoint.npy"))
# meta = np.load("checkpoint_meta.npz")

# lam = float(meta["lam"])
# global_iter = int(meta["global_iter"])
# rho_hist = meta["rho_hist"].tolist()
# cost_hist = meta["cost_hist"].tolist()
# gnorm_hist = meta["gnorm_hist"].tolist()


cost_window = deque(maxlen=stall_window) 
lam, global_iter = 10.0, 0

print(f"{'Iter':<8} | {'rho':<10} | {'Total Cost':<12} | {'gnorm':<10} | {'lam':<8}")
print("-" * 65)

try:
    for out_it in range(50):
        # Apply Hessian approximation to the full gradient later
        H_inv = jnp.array([1.0/(2*w_v), 1.0/(2*w_w)]) # Approximate Hessian for control
        cost_window.clear() 
        
        for epoch in range(maxIter):
            indices = np.random.permutation(num_agents)
            for blk in indices:
                # Current base cost and robustness
                rho_curr = total_rho(U)
                J_base = compute_full_objective1(U, lam)
                
                # Full gradient (Control + Penalty)
                full_grad = rho_grad_fun(U, lam)
                curr_gnorm = float(jnp.linalg.norm(full_grad))
                
                # Histories
                rho_hist.append(float(rho_curr))
                cost_hist.append(float(J_base))
                gnorm_hist.append(curr_gnorm)
                
                # UPDATE: Use the full_grad[blk] directly
                g_blk = full_grad[blk]
                d_blk = -g_blk * H_inv[:, None]
                slope = jnp.sum(g_blk * d_blk)
                
                # Line Search
                alpha, U_old_blk = alpha0, U[blk]
                for _ in range(maxLineIters):
                    U_trial = U.at[blk].set(U_old_blk + alpha * d_blk)
                    J_trial = compute_full_objective1(U_trial, lam)
                    if J_trial <= J_base + armijo_c * alpha * slope:
                        U, J_base = U_trial, J_trial
                        break
                    alpha *= armijo_beta

                cost_window.append(float(J_base))
            
            # Convergence Checks
            if len(cost_window) == stall_window:
                rel_change = abs(cost_window[0] - cost_window[-1]) / (abs(cost_window[0]) + 1e-9)
                if curr_gnorm < tol: break
                if rel_change < stall_tol and rho_curr < -0.01: break

            global_iter += 1
            if global_iter % 20 == 0:
                print(f"{global_iter:<8} | {rho_curr:>10.4f} | {float(J_base):>12.2f} | {curr_gnorm:>10.4f} | {lam:>8.1f}")

        if rho_curr >= -1e-3: 
            print("Satisfied!"); break
        
        # Increase penalty
        lam = min(lam * 2.0, 50000.0)

except KeyboardInterrupt: pass

# ===============================
# 4. Plots
# ===============================# Ensure all histories are numpy-friendly
rho_np = np.array(rho_hist)
cost_np = np.array(cost_hist)
gnorm_np = np.array(gnorm_hist)

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# Plot Robustness
axes[0].plot(rho_np, color='blue')
axes[0].set_title("Robustness ($\rho$)")
axes[0].grid(True)
axes[0].axhline(0, color='red', linestyle='--')

# Plot Total Cost (with safety check for log scale)
axes[1].plot(cost_np, color='purple')
axes[1].set_title("Total Objective ($J$)")
axes[1].grid(True)
if len(cost_np) > 0 and np.all(cost_np > 0):
    axes[1].set_yscale('log')

# Plot Gradient Norm
axes[2].plot(gnorm_np, color='green')
axes[2].set_title("Gradient Norm $||\nabla J||$")
axes[2].grid(True)
axes[2].set_yscale('log')

plt.tight_layout()
plt.show()

# 1. Generate final trajectories from the last U
Xall_final = np.array(vmap(simulate_agent_scan)(x0_list, U))

plt.figure(figsize=(10, 7))

# 2. Draw Obstacles
for c, r in zip(O_centers, r_obs):
    circle = patches.Circle(c, r, color='red', alpha=0.3, label='Obstacle' if c[0]==O_centers[0,0] else "")
    plt.gca().add_patch(circle)

# 3. Plot each agent's path
for i in range(num_agents):
    # Xall_final[agent_index, state_dim (x=0, y=1), time_step]
    plt.plot(Xall_final[i, 0, :], Xall_final[i, 1, :], label=f'Agent {i}', linewidth=1.5)
    # Plot starting point
    plt.scatter(Xall_final[i, 0, 0], Xall_final[i, 1, 0], marker='o')
    # Plot end point
    plt.scatter(Xall_final[i, 0, -1], Xall_final[i, 1, -1], marker='x')

plt.gca().set_aspect('equal')
plt.grid(True, linestyle='--', alpha=0.6)
plt.title(f"Final Trajectories (Final $\rho$: {rho_np[-1]:.4f})")
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
plt.show()


Iter     | rho        | Total Cost   | gnorm      | lam     
-----------------------------------------------------------------
20       |    -7.9056 |       637.83 |   895.2897 |     10.0
40       |    -7.7143 |       611.08 |   970.2448 |     10.0
60       |    -7.6297 |       598.74 |  1168.4952 |     10.0
80       |    -7.5538 |       587.86 |  2102.6323 |     10.0
100      |    -7.4695 |       576.02 |  1136.9495 |     10.0
120      |    -7.3763 |       563.35 |  1332.9041 |     10.0
140      |    -7.2736 |       549.51 |  1283.5775 |     10.0
160      |    -7.1495 |       533.25 |  1261.3041 |     10.0
180      |    -7.0055 |       514.80 |  1116.6654 |     10.0
200      |    -6.8351 |       493.50 |   871.5225 |     10.0
220      |    -6.4619 |       449.83 |   869.3541 |     10.0
240      |    -6.0145 |       401.57 |  3047.9937 |     10.0
260      |    -5.7593 |       375.59 |  1345.7072 |     10.0
280      |    -5.4961 |       350.48 |  1476.4568 |     10.0
300      |    -5.22

2680     |    -1.9780 |       206.63 |   146.1418 |     20.0
2700     |    -1.8925 |       203.82 |    78.7665 |     20.0
2720     |    -1.7989 |       201.39 |   124.3833 |     20.0
2740     |    -1.7440 |       199.81 |    73.2666 |     20.0
2760     |    -1.6983 |       198.67 |    58.2996 |     20.0
2780     |    -1.6489 |       197.63 |    75.2767 |     20.0
2800     |    -1.6094 |       196.85 |   118.7880 |     20.0
2820     |    -1.5817 |       196.31 |    58.0498 |     20.0
2840     |    -1.5524 |       195.78 |    36.7196 |     20.0
2860     |    -1.5311 |       195.40 |    38.5385 |     20.0
2880     |    -1.5060 |       195.04 |    40.3796 |     20.0
2900     |    -1.4915 |       194.81 |    51.2894 |     20.0
2920     |    -1.4783 |       194.60 |    54.7528 |     20.0
2940     |    -1.4636 |       194.38 |    44.1705 |     20.0
2960     |    -1.4480 |       194.13 |    41.8763 |     20.0
2980     |    -1.4380 |       193.98 |    42.5842 |     20.0
3000     |    -1.4257 | 

5380     |    -0.7824 |       207.57 |    65.8252 |     40.0
5400     |    -0.7821 |       207.55 |    50.5518 |     40.0
5420     |    -0.7819 |       207.54 |    51.2718 |     40.0
5440     |    -0.7816 |       207.52 |    68.4339 |     40.0
5460     |    -0.7814 |       207.50 |    67.8548 |     40.0
5480     |    -0.7808 |       207.48 |    67.7277 |     40.0
5500     |    -0.7808 |       207.47 |    52.4097 |     40.0
5520     |    -0.7806 |       207.45 |    50.9306 |     40.0
5540     |    -0.7805 |       207.44 |    50.0824 |     40.0
5560     |    -0.7794 |       207.39 |    70.2083 |     40.0
5580     |    -0.7785 |       207.35 |    86.9695 |     40.0
5600     |    -0.7773 |       207.34 |    37.0368 |     40.0
5620     |    -0.7768 |       207.33 |    36.8291 |     40.0
5640     |    -0.7764 |       207.31 |    37.2177 |     40.0
5660     |    -0.7763 |       207.30 |    85.4090 |     40.0
5680     |    -0.7761 |       207.29 |    37.4071 |     40.0
5700     |    -0.7759 | 

8080     |    -0.2199 |       221.48 |    79.0889 |    160.0
8100     |    -0.2199 |       221.46 |    80.7380 |    160.0
8120     |    -0.2197 |       221.44 |    78.3357 |    160.0
8140     |    -0.2196 |       221.42 |    78.1637 |    160.0
8160     |    -0.2195 |       221.40 |   105.9140 |    160.0
8180     |    -0.2194 |       221.38 |    75.9560 |    160.0
8200     |    -0.2191 |       221.34 |    76.9135 |    160.0
8220     |    -0.2191 |       221.32 |   106.3313 |    160.0
8240     |    -0.2190 |       221.30 |   106.4994 |    160.0
8260     |    -0.2190 |       221.29 |    76.7378 |    160.0
8280     |    -0.2189 |       221.27 |   107.0067 |    160.0
8300     |    -0.2189 |       221.24 |    75.2160 |    160.0
8320     |    -0.2188 |       221.22 |    78.4043 |    160.0
8340     |    -0.2187 |       221.21 |    75.9715 |    160.0
8360     |    -0.2185 |       221.19 |    76.2506 |    160.0
8380     |    -0.2185 |       221.17 |    76.4491 |    160.0
8400     |    -0.2186 | 

10780    |    -0.0075 |       225.92 |    69.2936 |   5120.0
10800    |    -0.0075 |       225.92 |    69.3310 |   5120.0
10820    |    -0.0075 |       225.91 |   112.5745 |   5120.0
10840    |    -0.0075 |       225.91 |   110.2925 |   5120.0
10860    |    -0.0075 |       225.91 |    70.5079 |   5120.0
10880    |    -0.0075 |       225.90 |   111.2840 |   5120.0
10900    |    -0.0074 |       225.90 |    72.3365 |   5120.0
10920    |    -0.0074 |       225.90 |   108.3185 |   5120.0
10940    |    -0.0074 |       225.89 |   108.1224 |   5120.0
10960    |    -0.0074 |       225.89 |    72.4179 |   5120.0
10980    |    -0.0074 |       225.89 |   106.9052 |   5120.0
11000    |    -0.0074 |       225.88 |    74.2439 |   5120.0
11020    |    -0.0074 |       225.88 |   106.0636 |   5120.0
11040    |    -0.0074 |       225.88 |   105.4297 |   5120.0
11060    |    -0.0074 |       225.87 |    74.0487 |   5120.0
11080    |    -0.0074 |       225.87 |   105.0453 |   5120.0
11100    |    -0.0074 | 

...

ValueError: 
Robustness ($ho$)
            ^
ParseException: Expected end of text, found '$'  (at char 12), (line:1, col:13)

Error in callback <function _draw_all_if_interactive at 0x1481bfa60> (for post_execute), with arguments args (),kwargs {}:


ValueError: 
Robustness ($ho$)
            ^
ParseException: Expected end of text, found '$'  (at char 12), (line:1, col:13)

ValueError: 
Robustness ($ho$)
            ^
ParseException: Expected end of text, found '$'  (at char 12), (line:1, col:13)

<Figure size 1800x400 with 3 Axes>

In [3]:
np.savez(
    "checkpoint_meta.npz",
    lam=float(lam),
    global_iter=int(global_iter),
    rho_hist=np.array(rho_hist),
    cost_hist=np.array(cost_hist),
    gnorm_hist=np.array(gnorm_hist)
)

import numpy as np
np.save("U_checkpoint.npy", np.array(U))


In [5]:
Xall_final = np.array(vmap(simulate_agent_scan)(x0_list, U))

In [15]:
conda install tikzplotlib

Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - tikzplotlib

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [13]:
import tikzplotlib

Xall_final = np.array(vmap(simulate_agent_scan)(x0_list, U))

plt.figure(figsize=(10, 7))

# Obstacles
for idx, (c, r) in enumerate(zip(np.array(O_centers), np.array(r_obs))):
    circle = patches.Circle(
        c,
        float(r),
        color='red',
        alpha=0.3,
        label='Obstacle' if idx == 0 else ""
    )
    plt.gca().add_patch(circle)

# Agent trajectories
for i in range(num_agents):
    plt.plot(
        Xall_final[i, 0, :],
        Xall_final[i, 1, :],
        label=f'Agent {i}',
        linewidth=1.5
    )
    plt.scatter(Xall_final[i, 0, 0],  Xall_final[i, 1, 0], marker='o')
    plt.scatter(Xall_final[i, 0, -1], Xall_final[i, 1, -1], marker='x')

plt.gca().set_aspect('equal')
plt.grid(True, linestyle='--', alpha=0.6)
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title(f"Final Trajectories (Final $\\rho$: {rho_hist[-1]:.4f})")
plt.legend(ncol=2, fontsize=9)
plt.tight_layout()
plt.show()

tikzplotlib.save("trajectories.tex")

ImportError: cannot import name 'common_texification' from 'matplotlib.backends.backend_pgf' (/Applications/miniconda3/envs/jax_env/lib/python3.12/site-packages/matplotlib/backends/backend_pgf.py)

In [16]:
import numpy as np

# Ensure Xall_final is a NumPy array
Xall_final = np.array(Xall_final)

num_agents = Xall_final.shape[0]

with open("trajectories_manual.tex", "w") as f:
    f.write("\\begin{tikzpicture}\n")
    f.write("  \\begin{axis}[axis equal, grid=both, xlabel={X}, ylabel={Y}]\n")
    
    # Plot agent trajectories
    for i in range(num_agents):
        x = Xall_final[i, 0, :]
        y = Xall_final[i, 1, :]
        coords = " ".join([f"({x[k]:.3f},{y[k]:.3f})" for k in range(len(x))])
        f.write(f"    \\addplot+[mark=none, thick] coordinates {{{coords}}};\n")
        f.write(f"    \\addlegendentry{{Agent {i}}}\n")

    # Plot obstacles
    for c, r in zip(O_centers, r_obs):
        f.write(
            f"    \\draw[red, fill=red!20] ({float(c[0])},{float(c[1])}) circle ({float(r)});\n"
        )

    f.write("  \\end{axis}\n")
    f.write("\\end{tikzpicture}\n")


In [10]:
pip install tikzplotlib

/Applications/miniconda3/envs/jax_env/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tikzplotlib] [tikzplotlib]
Note: you may need to restart the kernel to use updated packages.


In [11]:
import tikzplotlib




ImportError: cannot import name 'common_texification' from 'matplotlib.backends.backend_pgf' (/Applications/miniconda3/envs/jax_env/lib/python3.12/site-packages/matplotlib/backends/backend_pgf.py)